In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from json import load, dump as save
import re

clean = re.compile(r'\n|\s{2,}').sub
id = lambda item: item.get_attribute('data-id')
price = lambda item: int(item.find_element_by_xpath('.//span[@itemprop="price"]').text)
url = 'https://www.splashfashions.com/ae/en/search?price=0,40&q=allCategories:men-regular:badge.title.en:Sale:index:price'

def foreach_item(execute):
    for page in range(2, 22):
        [execute(item) for item in br.find_elements_by_class_name('product-item')]
        try: br.find_element_by_class_name('next').click()
        except: break
        if page < 21: WebDriverWait(br, 10).until(EC.text_to_be_present_in_element((By.CLASS_NAME, 'current'), str(page)))

def get_old_discounts():
    try: return load(open('old.json'))
    except: old = {}
    foreach_item(lambda item: old.setdefault(id(item), price(item)))
    save(old, open('old.json', 'w+'))
    return old